### DOES ATTENDING CATHOLIC SCHOOL HELP IMPROVE MATHS SCORE?

The main goal of this notebook is to introduce the notion of confounders in causal inference and provide a way to control them in a causal effect estimation.

We delve into a longitudinal study carried out by the University of Michigan to understand whether catholic/private schools are more beneficial for the student's math education. For more information: https://www.icpsr.umich.edu/web/ICPSR/studies/4075.

In [4]:
# !pip install skimpy

In [5]:
# pip install -U -q statsmodels

In [6]:
import statsmodels.formula.api as smf
import pandas as pd
from skimpy import skim

#### DATA LOADING

In [7]:
df = pd.read_csv("data_ecls.csv")
df.head()

,childid,catholic,race,race_white,race_black,race_hispanic,race_asian,NumPlace,Age_Mom,Age_Dad,...,BelowHighSchool_Dad,BelowHighSchool_Mom,JobScore_Mom,JobScore_Dad,IncomeCat,Income,Poverty,FoodStamp,Score_t,Score_sd
0,0001002C,0,"WHITE, NON-HISPANIC",1,0,0,0,1.0,47.0,45.0,...,0.0,0.0,53.50,77.5,"$50,001 TO $75,000",62500.5,0.0,0.0,60.043,0.981753
1,0001004C,0,"WHITE, NON-HISPANIC",1,0,0,0,1.0,41.0,48.0,...,0.0,0.0,34.95,53.5,"$40,001 TO $50,000",45000.5,0.0,0.0,56.280,0.594378
2,0001005C,0,"WHITE, NON-HISPANIC",1,0,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.791,0.338152
3,0001010C,0,"WHITE, NON-HISPANIC",1,0,0,0,1.0,43.0,55.0,...,0.0,0.0,63.43,53.5,"$50,001 TO $75,000",62500.5,0.0,0.0,55.272,0.490611
4,0001011C,1,"WHITE, NON-HISPANIC",1,0,0,0,1.0,38.0,39.0,...,0.0,0.0,53.50,53.5,"$75,001 TO $100,000",87500.5,0.0,0.0,64.604,1.451278


#### DESCRIPTIVE STATS

In [8]:
skim(df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 11078  │ │ float64     │ 12    │                                                          │
│ │ Number of columns │ 22     │ │ string      │ 5     │                                                          │
│ └───────────────────┴────────┘ │ int32       │ 5     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name    ┃ NA   ┃ NA %  ┃ mean       ┃ sd     ┃ p0     ┃ p25     ┃ p50    ┃ p75    ┃ p100   ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩  │
│ │ catholic       │    0 │     0 │     0.1363 │ 0.3431 │      0 │       0 │      0 │      0 │      1 │ ▇    ▁ │  │
│ │ race_white     │    0 │     0 │     0.5792 │ 0.4937 │      0 │       0 │      1 │      1 │      1 │ ▆    ▇ │  │
│ │ race_black     │    0 │     0 │     0.1252 │  0.331 │      0 │       0 │      0 │      0 │      1 │ ▇    ▁ │  │
│ │ race_hispanic  │    0 │     0 │     0.1745 │ 0.3795 │      0 │       0 │      0 │      0 │      1 │ ▇    ▂ │  │
│ │ race_asian     │    0 │     0 │    0.06445 │ 0.2456 │      0 │       0 │      0 │      0 │      1 │ ▇    ▁ │  │
│ │ NumPlace       │ 1575 │ 14.22 │      1.127 │ 0.3886 │      1 │       1 │      1 │      1 │      6 │   ▇▁   │  │
│ │ Age_Mom        │ 1615 │ 14.58 │      37.85 │   6.44 │     19 │      33 │     38 │     42 │     76 │  ▁▇▇▁  │  │
│ │ Age_Dad        │ 3232 │ 29.17 │      40.64 │  6.882 │     22 │      36 │     40 │     45 │     81 │  ▁▇▆▁  │  │
│ │ BelowHighSchoo │ 3210 │ 28.98 │     0.4527 │ 0.4978 │      0 │       0 │      0 │      1 │      1 │ ▇    ▇ │  │
│ │ l_Dad          │      │       │            │        │        │         │        │        │        │        │  │
│ │ BelowHighSchoo │ 1592 │ 14.37 │     0.4296 │  0.495 │      0 │       0 │      0 │      1 │      1 │ ▇    ▆ │  │
│ │ l_Mom          │      │       │            │        │        │         │        │        │        │        │  │
│ │ JobScore_Mom   │ 3755 │  33.9 │      43.79 │  11.22 │   29.6 │   34.95 │  38.18 │   53.5 │   77.5 │ ▇▆▃▁▃  │  │
│ │ JobScore_Dad   │ 3571 │ 32.24 │       43.3 │  10.84 │   29.6 │   35.78 │  39.18 │   53.5 │   77.5 │ ▇▅▅▁▁  │  │
│ │ Income         │ 1374 │  12.4 │      58800 │  45150 │   5000 │   27500 │  45000 │  87500 │ 200000 │ ▇▇▃ ▂▁ │  │
│ │ Poverty        │ 1374 │  12.4 │     0.1931 │ 0.3948 │      0 │       0 │      0 │      0 │      1 │ ▇    ▂ │  │
│ │ FoodStamp      │ 1524 │ 13.76 │     0.1129 │ 0.3165 │      0 │       0 │      0 │      0 │      1 │ ▇    ▁ │  │
│ │ Score_t        │    0 │     0 │      50.51 │  9.714 │  15.92 │   44.09 │  51.21 │  57.14 │  83.86 │  ▂▇▇▂  │  │
│ │ Score_sd       │    0 │     0 │ -3.348e-16 │      1 │ -3.561 │ -0.6609 │ 0.0723 │ 0.6831 │  3.434 │  ▂▇▇▂  │  │
│ └────────────────┴──────┴───────┴────────────┴────────┴────────┴─────────┴────────┴────────┴────────┴────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳

In [9]:
# pip install qolmat

In [10]:
# pip install --force-reinstall -U setuptools

In [11]:
from qolmat.imputations import imputers

In [12]:
from qolmat.benchmark import comparator, missing_patterns


In [13]:
from qolmat.imputations import preprocessing
from qolmat.imputations.imputers import ImputerRegressor
from sklearn.pipeline import Pipeline

We're using Qolmat to imput the missing data

In [14]:
num_cols = ["NumPlace", "Age_Mom", "Age_Dad", "BelowHighSchool_Dad", "BelowHighSchool_Mom", "JobScore_Mom", "JobScore_Dad",
           "Income", "Poverty", "FoodStamp"]
cat_cols = ["Ed_Dad", "Ed_Mom", "IncomeCat"]

In [15]:
imputer_simple = imputers.ImputerSimple()

In [16]:
imputer_rpca = imputers.ImputerRpcaNoisy()
ohe = preprocessing.OneHotEncoderProjector(
    handle_unknown="ignore",
    handle_missing="return_nan",
    use_cat_names=True,
    cols=cat_cols,
)
bt = preprocessing.BinTransformer(cols=num_cols)
wrapper = Pipeline(steps=[("OneHotEncoder", ohe), ("BinTransformer", bt)])
imputer_wrap_rpca = preprocessing.WrapperTransformer(imputer_rpca, wrapper)

In [17]:
pipestimator = preprocessing.make_robust_MixteHGB(avoid_new=True)
imputer_hgb = ImputerRegressor(estimator=pipestimator, handler_nan="none")
imputer_wrap_hgb = preprocessing.WrapperTransformer(imputer_hgb, bt)

In [18]:
dict_imputers = {
    "Simple": imputer_simple,
    "HGB": imputer_wrap_hgb,
    "RPCA": imputer_wrap_rpca,
}
cols_to_impute = ["NumPlace", "Age_Mom", "Age_Dad", "BelowHighSchool_Dad", "BelowHighSchool_Mom", "JobScore_Mom", "JobScore_Dad",
           "Income", "Poverty", "FoodStamp", "Ed_Dad", "Ed_Mom", "IncomeCat"]
ratio_masked = 0.1
generator_holes = missing_patterns.UniformHoleGenerator(
    n_splits=2,
    subset=cols_to_impute,
    ratio_masked=ratio_masked,
    sample_proportional=False,
)
metrics = ["rmse", "accuracy"]

comparison = comparator.Comparator(
    dict_imputers,
    cols_to_impute,
    generator_holes=generator_holes,
    metrics=metrics,
    max_evals=2,
)
results = comparison.compare(df[["NumPlace", "Age_Mom", "Age_Dad", "BelowHighSchool_Dad", "BelowHighSchool_Mom", "JobScore_Mom", "JobScore_Dad",
           "Income", "Poverty", "FoodStamp", "Ed_Dad", "Ed_Mom", "IncomeCat"]])

Testing model: Simple...

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\qolmat\benchmark\missing_patterns.py:203: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_mask[col].iloc[indices] = True
c:\Users\DELL\AppData\Local\Programs\Python\Pyt

done.
Testing model: HGB...

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: '

done.
Testing model: RPCA...

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: '

done.


c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: '

In [19]:
pip install Jinja2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
results.loc["rmse"].style.highlight_min(color="blue", axis=1)


,Simple,HGB,RPCA
NumPlace,0.401374,0.389291,0.373158
Age_Mom,6.424053,5.105605,5.151714
Age_Dad,7.090031,4.993232,4.883965
BelowHighSchool_Dad,0.676780,0.175155,0.172080
BelowHighSchool_Mom,0.662882,0.180096,0.180253
JobScore_Mom,12.456851,9.559773,9.802122
JobScore_Dad,11.384039,8.575160,8.846237
Income,46471.940318,12203.876845,11715.905537
Poverty,0.438431,0.212387,0.241948
FoodStamp,0.343845,0.304862,0.310010


In [21]:
results.loc["accuracy"].style.highlight_max(color="green", axis=1)


,Simple,HGB,RPCA
NumPlace,0.898466,0.879964,0.891245
Age_Mom,0.057310,0.097924,0.099278
Age_Dad,0.067238,0.088448,0.102437
BelowHighSchool_Dad,0.541968,0.969314,0.970217
BelowHighSchool_Mom,0.560469,0.967509,0.967509
JobScore_Mom,0.275722,0.043321,0.036101
JobScore_Dad,0.080325,0.092509,0.091155
Income,0.124097,0.903881,0.923285
Poverty,0.807762,0.954874,0.941336
FoodStamp,0.881769,0.907040,0.903881


In [22]:
import matplotlib
import matplotlib.pyplot as plt

In [23]:
cols_to_impute = ["NumPlace", "Age_Mom", "Age_Dad", "BelowHighSchool_Dad", "BelowHighSchool_Mom", "JobScore_Mom", "JobScore_Dad",
           "Income", "Poverty", "FoodStamp", "Ed_Dad", "Ed_Mom", "IncomeCat"]

In [26]:
dic_1 = {name: imp for name, imp in dict_imputers.items()}

In [30]:
df_imputed = df.copy()

In [31]:
df_imputed = dic_1["HGB"].fit_transform(df_imputed)

In [32]:
df_imputed.to_csv('data_ecls_imputed.csv', index=False)  

In [33]:
skim(df_imputed)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 11078  │ │ float64     │ 12    │                                                          │
│ │ Number of columns │ 22     │ │ string      │ 5     │                                                          │
│ └───────────────────┴────────┘ │ int32       │ 5     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━┳━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name       ┃ NA ┃ NA % ┃ mean       ┃ sd     ┃ p0     ┃ p25     ┃ p50    ┃ p75    ┃ p100   ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━╇━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩  │
│ │ catholic          │  0 │    0 │     0.1363 │ 0.3431 │      0 │       0 │      0 │      0 │      1 │ ▇    ▁ │  │
│ │ race_white        │  0 │    0 │     0.5792 │ 0.4937 │      0 │       0 │      1 │      1 │      1 │ ▆    ▇ │  │
│ │ race_black        │  0 │    0 │     0.1252 │  0.331 │      0 │       0 │      0 │      0 │      1 │ ▇    ▁ │  │
│ │ race_hispanic     │  0 │    0 │     0.1745 │ 0.3795 │      0 │       0 │      0 │      0 │      1 │ ▇    ▂ │  │
│ │ race_asian        │  0 │    0 │    0.06445 │ 0.2456 │      0 │       0 │      0 │      0 │      1 │ ▇    ▁ │  │
│ │ NumPlace          │  0 │    0 │      1.109 │ 0.3627 │      1 │       1 │      1 │      1 │      6 │   ▇▁   │  │
│ │ Age_Mom           │  0 │    0 │      37.84 │  6.031 │     19 │      34 │     38 │     41 │     76 │  ▁▇▇▁  │  │
│ │ Age_Dad           │  0 │    0 │      40.82 │  6.554 │     22 │      37 │     40 │     44 │     81 │  ▁▇▅▁  │  │
│ │ BelowHighSchool_D │  0 │    0 │     0.3215 │ 0.4671 │      0 │       0 │      0 │      1 │      1 │ ▇    ▃ │  │
│ │ ad                │    │      │            │        │        │         │        │        │        │        │  │
│ │ BelowHighSchool_M │  0 │    0 │     0.3678 │ 0.4822 │      0 │       0 │      0 │      1 │      1 │ ▇    ▅ │  │
│ │ om                │    │      │            │        │        │         │        │        │        │        │  │
│ │ JobScore_Mom      │  0 │    0 │       44.9 │  10.62 │   29.6 │   35.78 │  38.18 │   53.5 │   77.5 │ ▇▇▅▃▃  │  │
│ │ JobScore_Dad      │  0 │    0 │      44.32 │  9.854 │   29.6 │   35.92 │   39.2 │   53.5 │   77.5 │ ▇▇▇▁▁  │  │
│ │ Income            │  0 │    0 │      54300 │  43920 │   5000 │   22500 │  37500 │  62500 │ 200000 │ ▇▆▂ ▂  │  │
│ │ Poverty           │  0 │    0 │     0.1692 │ 0.3749 │      0 │       0 │      0 │      0 │      1 │ ▇    ▂ │  │
│ │ FoodStamp         │  0 │    0 │    0.09921 │  0.299 │      0 │       0 │      0 │      0 │      1 │ ▇    ▁ │  │
│ │ Score_t           │  0 │    0 │      50.51 │  9.714 │  15.92 │   44.09 │  51.21 │  57.14 │  83.86 │  ▂▇▇▂  │  │
│ │ Score_sd          │  0 │    0 │ -3.348e-16 │      1 │ -3.561 │ -0.6609 │ 0.0723 │ 0.6831 │  3.434 │  ▂▇▇▂  │  │
│ └───────────────────┴────┴──────┴────────────┴────────┴────────┴─────────┴────────┴────────┴────────┴────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━

### Regression Analysis

In [35]:
mod_1 = smf.ols("Score_t ~ catholic", data=df).fit()
print(mod_1.summary())

                            OLS Regression Results                            
Dep. Variable:                Score_t   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     66.10
Date:                Wed, 20 Nov 2024   Prob (F-statistic):           4.75e-16
Time:                        14:58:40   Log-Likelihood:                -40872.
No. Observations:               11078   AIC:                         8.175e+04
Df Residuals:                   11076   BIC:                         8.176e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     50.2090      0.099    507.064      0.0

In [36]:
mod_2 = smf.ols("Score_t ~ catholic", data=df_imputed).fit()
print(mod_2.summary())


                            OLS Regression Results                            
Dep. Variable:                Score_t   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     66.10
Date:                Wed, 20 Nov 2024   Prob (F-statistic):           4.75e-16
Time:                        14:58:42   Log-Likelihood:                -40872.
No. Observations:               11078   AIC:                         8.175e+04
Df Residuals:                   11076   BIC:                         8.176e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     50.2090      0.099    507.064      0.0

In [40]:
mod_3 = smf.ols("Score_t ~ catholic + race_white+ race_black + race_hispanic + race_asian", data=df).fit()
print(mod_3.summary())


                            OLS Regression Results                            
Dep. Variable:                Score_t   R-squared:                       0.106
Model:                            OLS   Adj. R-squared:                  0.106
Method:                 Least Squares   F-statistic:                     263.6
Date:                Wed, 20 Nov 2024   Prob (F-statistic):          3.91e-267
Time:                        15:07:18   Log-Likelihood:                -40282.
No. Observations:               11078   AIC:                         8.058e+04
Df Residuals:                   11072   BIC:                         8.062e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        47.2484      0.367    128.653

In [41]:
mod_4 = smf.ols("Score_t ~ catholic + race_white+ race_black + race_hispanic + race_asian", data=df_imputed).fit()
print(mod_4.summary())


                            OLS Regression Results                            
Dep. Variable:                Score_t   R-squared:                       0.106
Model:                            OLS   Adj. R-squared:                  0.106
Method:                 Least Squares   F-statistic:                     263.6
Date:                Wed, 20 Nov 2024   Prob (F-statistic):          3.91e-267
Time:                        15:07:20   Log-Likelihood:                -40282.
No. Observations:               11078   AIC:                         8.058e+04
Df Residuals:                   11072   BIC:                         8.062e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        47.2484      0.367    128.653

In [42]:
mod_5 = smf.ols("Score_t ~ catholic + race_white+ race_black + race_hispanic + race_asian  + NumPlace", data=df).fit()
print(mod_5.summary())

                            OLS Regression Results                            
Dep. Variable:                Score_t   R-squared:                       0.105
Model:                            OLS   Adj. R-squared:                  0.105
Method:                 Least Squares   F-statistic:                     186.2
Date:                Wed, 20 Nov 2024   Prob (F-statistic):          4.86e-225
Time:                        15:08:36   Log-Likelihood:                -34499.
No. Observations:                9503   AIC:                         6.901e+04
Df Residuals:                    9496   BIC:                         6.906e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        49.0908      0.486    100.948

In [43]:
mod_6 = smf.ols("Score_t ~ catholic + race_white+ race_black + race_hispanic + race_asian  + NumPlace", data=df_imputed).fit()
print(mod_6.summary())

                            OLS Regression Results                            
Dep. Variable:                Score_t   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.108
Method:                 Least Squares   F-statistic:                     223.9
Date:                Wed, 20 Nov 2024   Prob (F-statistic):          8.22e-271
Time:                        15:08:45   Log-Likelihood:                -40271.
No. Observations:               11078   AIC:                         8.056e+04
Df Residuals:                   11071   BIC:                         8.061e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        48.5580      0.458    105.920

In [45]:
mod_7 = smf.ols("Score_t ~ catholic + race_white+ race_black + race_hispanic + race_asian  + NumPlace + Income", data=df).fit()
print(mod_7.summary())

                            OLS Regression Results                            
Dep. Variable:                Score_t   R-squared:                       0.171
Model:                            OLS   Adj. R-squared:                  0.170
Method:                 Least Squares   F-statistic:                     279.7
Date:                Wed, 20 Nov 2024   Prob (F-statistic):               0.00
Time:                        15:09:56   Log-Likelihood:                -34137.
No. Observations:                9503   AIC:                         6.829e+04
Df Residuals:                    9495   BIC:                         6.835e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        45.9339      0.482     95.281

In [44]:
mod_8 = smf.ols("Score_t ~ catholic + race_white+ race_black + race_hispanic + race_asian  + NumPlace + Income", data=df_imputed).fit()
print(mod_8.summary())

                            OLS Regression Results                            
Dep. Variable:                Score_t   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.172
Method:                 Least Squares   F-statistic:                     328.7
Date:                Wed, 20 Nov 2024   Prob (F-statistic):               0.00
Time:                        15:09:37   Log-Likelihood:                -39859.
No. Observations:               11078   AIC:                         7.973e+04
Df Residuals:                   11070   BIC:                         7.979e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        45.7464      0.452    101.185

In [46]:
mod_9 = smf.ols("Score_t ~ catholic + race_white+ race_black + race_hispanic + race_asian  + NumPlace + Income + JobScore_Mom + JobScore_Dad", data=df).fit()
print(mod_9.summary())

                            OLS Regression Results                            
Dep. Variable:                Score_t   R-squared:                       0.154
Model:                            OLS   Adj. R-squared:                  0.153
Method:                 Least Squares   F-statistic:                     110.3
Date:                Wed, 20 Nov 2024   Prob (F-statistic):          1.08e-190
Time:                        15:11:03   Log-Likelihood:                -19484.
No. Observations:                5467   AIC:                         3.899e+04
Df Residuals:                    5457   BIC:                         3.905e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        40.0474      0.866     46.237

In [47]:
mod_10 = smf.ols("Score_t ~ catholic + race_white+ race_black + race_hispanic + race_asian  + NumPlace + Income + JobScore_Mom + JobScore_Dad", data=df_imputed).fit()
print(mod_10.summary())

                            OLS Regression Results                            
Dep. Variable:                Score_t   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.181
Method:                 Least Squares   F-statistic:                     272.3
Date:                Wed, 20 Nov 2024   Prob (F-statistic):               0.00
Time:                        15:11:22   Log-Likelihood:                -39797.
No. Observations:               11078   AIC:                         7.961e+04
Df Residuals:                   11068   BIC:                         7.969e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        41.1636      0.635     64.788

In [48]:
!pip install -U -q stargazer


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
from stargazer.stargazer import Stargazer

In [50]:
gazer_raw = [mod_1, mod_3, mod_5, mod_7, mod_9]
gazer_imputed = [mod_2, mod_4, mod_6, mod_8, mod_10]

Stargazer(gazer_raw)

Using the raw data, Going to a catholic school has a significant impact on the math score of students but the impact if nnot consistent(positive and negative) when we control for more variables. 

In [51]:
Stargazer(gazer_imputed)

Using the cleaned data, Going to a catholic school has a significant impact on the math score of students but the impact if nnot consistent(positive and negative) when we control for more variables. Though, the Model_4(controling for ``income``) the impact is not statistically significant.

We could try to drop all the NAs values(reducing significantly our dataset) but we'll have the same inconsistent impact effect of the Treatment(``Catholic``) onto the dependent variable(``Score_t``). This shows the real problem confounfing factors (``income`` or ``race``) have in biaising estimates.

One has to make sure to find a way to account for these properly (in our case using a multivariate regression, but we cannot control all the existing confounders 😪) using for instance Randomized Control Trials, etc. : Here's a link to learn more about confounding in causal inference https://en.wikipedia.org/wiki/Confounding 